In [1]:
newestModel = True #If this is set to true it will use the newest model in the models folder and ignore modelToTest
modelToTest = ''
pathToModelFolder = '~/SPGenerations/Models/'
pathToDataFolder = '~/SPDataset/smallDataset/S2DS245-1014/'
#a folder inthe models folder contains the following files: model.h5, tokenizer.json commitHash.txt (For SCA-Tokenizer)
useCurrentSCATokenizer = True #If this is set to true it will use the current SCA-Token in the SCA-Token folder and ignore the modelToTest











In [5]:
import os
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import subprocess
import time
import datetime
import random
import math
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import tokenizer_from_json


def homePath(path):
    return os.path.join(os.path.expanduser("~"), path.strip("~/"))

pathToModelFolder = homePath(pathToModelFolder)
pathToDataFolder = homePath(pathToDataFolder)



In [9]:
#File Loading

if newestModel:
    models = os.listdir(pathToModelFolder)
    models.sort()
    modelToTest = models[-1]
    print("Using newest model: " + modelToTest)
else:
    print("Using model: " + modelToTest)
model = load_model(pathToModelFolder + '/' +modelToTest + '/model.h5')
print (model.summary())

#load tokenizer from json that used the .tokenizer.save() function
with open(pathToModelFolder + '/' + modelToTest + '/tokenizer.json') as f:
    tokenizer = tokenizer_from_json(f.read())


Using newest model: 2023-10-17 18:59:54.067245
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 1024)         10240000  
                                                                 
 conv1d (Conv1D)             (None, 398, 2048)         6293504   
                                                                 
 global_max_pooling1d (Glob  (None, 2048)              0         
 alMaxPooling1D)                                                 
                                                                 
 reshape (Reshape)           (None, 1, 2048)           0         
                                                                 
 bidirectional (Bidirection  (None, 1, 4096)           67125248  
 al)                                                             
                                                                 
 dropout 

TypeError: the JSON object must be str, bytes or bytearray, not builtin_function_or_method

In [ ]:
pathToSCATokenizerFolder = os.getcwd() + '/SCA-Tokenizer/'
if (not useCurrentSCATokenizer):
    #TODO: Configure Git clone on the commit hash

